In [1]:
import os
os.chdir('..')

In [2]:
import torch
import scvi
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt

sc.set_figure_params(dpi=100, frameon=False, color_map='Reds', facecolor=None)
sc.logging.print_header()

/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:2: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion
Global seed set to 0
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)


scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.22.4 scipy==1.9.1 pandas==1.4.4 scikit-learn==1.1.2 statsmodels==0.13.2 pynndescent==0.5.7


/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
hlca_path = 'datasets/hlca.h5ad'
adata = sc.read(hlca_path)
adata

AnnData object with n_obs × n_vars = 584944 × 2000
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'subject_ID', 'subject_ID_as_published', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'Processing_site', 'dataset', 'anatomical_region_level_1', 'anatomical_region_level_2', 'anatomical_region_level_3', 'anatomical_region_highest_res', 'age', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_ne

In [5]:
adata.var.to_csv('datasets/hlca_hvgs.csv')

In [7]:
marker_df = pd.read_csv('/home/icb/yuge.ji/projects/HLCA_reproducibility/notebooks/3_atlas_extension/markergenes.csv', index_col=0)

In [8]:
marker_dict = {}                                                                                                                                                                                                                                        
for i in range(0, marker_df.shape[1], 3):                                                                                                                                                                                                               
    ct = marker_df.columns[i].split('_')[0]                                                                                                                                                                                                             
    query_res = marker_df[[c for c in marker_df.columns if ct in c]][[f'{ct}_marker', f'{ct}_marker_for']].dropna().values                                                                                                                              
    for value, key in query_res:                                                                                                                                                                                                                        
        marker_dict.setdefault(key, []).append(value) 
marker_dict = {k.replace(' (poss. lowly expressed, non-unique)', ''):v for k, v in marker_dict.items()}
marker_dict = {k.replace(' (poss. lowly expressed, non-unique, non-universal)', ''):v for k, v in marker_dict.items()}

Mapping by Lisa.

In [9]:
mapping = {
 'Arterial EC' :['EC arterial'],
 'B cell lineage':['B cells', "Plasma cells"],
 'Basal':['Basal resting', 'Suprabasal', 'Hillock-like'],
#  'Bronchial Vessel 1':"EXCLUDE!",
#  'Bronchial Vessel 2':"EXCLUDE!",
 'Capillary':['EC aerocyte capillary', 'EC general capillary'],
 'Ciliated':['Deuterosomal', 'Multiciliated (nasal)', 'Multiciliated (non-nasal)'],
 'Dendritic cells':['DC1', 'DC2', 'Migratory DCs', 'Plasmacytoid DCs'],
#  'KRT5- KRT17+ epithelial':"EXCLUDE!",
 'Macrophages':['Alveolar macrophages',
 'Alveolar Mph CCL3+',
 'Alveolar Mph MT-positive',
 'Alveolar Mph proliferating',
#  'Monocyte-derived Mph',
 'Interstitial Mph perivascular'],
#  'Megakaryocytes':"EXCLUDE!",
 'Monocytes':['Classical monocytes', 'Non-classical monocytes'],
#  'Neutrophilic':"EXCLUDE!",
 'Non-T/B cells':['NK cells'],
#  'Proliferating cells':"EXCLUDE!",
 'Rare':['Ionocyte', 'Tuft', 'Neuroendocrine'],
 'Secretory': ['Club (non-nasal)',
 'Club (nasal)',
 'Goblet (nasal)',
 'Goblet (bronchial)',
 'Goblet (subsegmental)',
 'AT0',
 'pre-TB secretory'],
 'Smooth Muscle':['Smooth muscle', 'Fibromyocytes', 'SM activated stress response'],
#  'Squamous':"EXCLUDE!",
 'Submucosal Secretory':['SMG serous (nasal)', 'SMG serous (bronchial)', 'SMG mucous', 'SMG duct'],
 'T cell lineage':[
     'CD4 T cells',
     'CD8 T cells',
     'T cells proliferating'],
 'Venous':['EC venous systemic', 'EC venous pulmonary'],
#  'unlabeled':"EXCLUDE!"
}

In [10]:
new_marker_dict = {}
for coarse_ct, ct_list in mapping.items():
    new_marker_dict[coarse_ct] = [g for c in ct_list for g in marker_dict[c]]

In [11]:
marker_dict = {**marker_dict, **new_marker_dict}

In [12]:
df = pd.DataFrame.from_dict(marker_dict, orient='index').T
df.to_csv('datasets/hlca_marker_dict.csv')
df

,Epithelial,Airway epithelium,AT0,Alveolar epithelium,AT1,AT2 proliferating,AT2,Stroma,Fibroblast lineage,Adventitial fibroblasts,...,Dendritic cells,Macrophages,Monocytes,Non-T/B cells,Rare,Secretory,Smooth Muscle,Submucosal Secretory,T cell lineage,Venous
0,FXYD3,IGFBP2,SFTPB,HOPX,CLIC5,CENPW,MFSD2A,COL1A2,LUM,MFAP5,...,XCR1,CYP27A1,S100A12,GNLY,BSND,CYP2F1,MYH11,STATH,CD40LG,TPD52L1
1,EPCAM,SERPINF1,SCGB3A2,SFTA2,SPOCK2,CDKN3,C8orf4,DCN,COL6A3,SCARA5,...,CLEC9A,MARCO,FCN1,SPON2,IGF1,SCGB3A1,TINAGL1,ODAM,TNFRSF25,ZNF385D
2,ELF3,TSPAN1,SFTA2,SLC39A8,TIMP3,BIRC5,C11orf96,MFAP4,CFD,PI16,...,CLNK,FABP4,RNASE2,FCGR3A,CLCNKB,BPIFB1,PLN,MUC7,CD28,OLFM1
3,FXYD3,IGFBP2,None,HOPX,None,None,None,COL1A2,LUM,None,...,PKIB,CCL20,MTSS1,None,DAB1,SERPINB3,MYH11,RP11-1143G9.4,CD4,CPE
4,EPCAM,SERPINF1,None,SFTA2,None,None,None,DCN,COL6A3,None,...,CLEC10A,FAM89A,LILRA5,None,CELF3,TCN1,TINAGL1,S100A1,TRAT1,C7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,EPCAM,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
62,ELF3,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
63,FXYD3,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
64,EPCAM,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


Reload to filter:

In [13]:
from scripts import hlca_markers
marker_dict = hlca_markers()

In [14]:
print(len(set(adata.obs.scanvi_label.unique()) - set(marker_dict.keys())))
set(adata.obs.scanvi_label.unique()) - set(marker_dict.keys())

8


{'Bronchial Vessel 1',
 'Bronchial Vessel 2',
 'KRT5- KRT17+ epithelial',
 'Megakaryocytes',
 'Neutrophilic',
 'Proliferating cells',
 'Squamous',
 'unlabeled'}

In [15]:
len(marker_dict.keys())

81

In [16]:
min_cells = 100
indices = []
cts_included = set()
for dataset in adata.obs.dataset.unique():
    subset = adata[adata.obs.dataset == dataset]
    for ct in subset.obs.scanvi_label.unique():
        if ct in marker_dict.keys() and subset.obs.scanvi_label.value_counts()[ct] > min_cells:
            cts_included.add(ct)
            indices.append(np.random.choice(subset[subset.obs.scanvi_label == ct].obs.index, min_cells, replace=False))
(len(indices))
len(cts_included)

21

In [142]:
new_adata = adata[np.array(indices).flatten()]
new_adata.shape

(14500, 2000)

In [143]:
new_adata.write('datasets/hlca_subset.h5ad')

In [144]:
new_adata.obs.scanvi_label.value_counts()

Ciliated                1300
Secretory               1100
Monocytes               1000
T cell lineage          1000
Basal                   1000
Dendritic cells         1000
Macrophages              900
Mast cells               900
AT2                      800
B cell lineage           700
Fibroblast lineage       700
Lymphatic EC             700
Submucosal Secretory     600
AT1                      600
Smooth Muscle            500
Venous                   400
Non-T/B cells            300
Rare                     300
Arterial EC              300
Mesothelium              200
Capillary                200
Name: scanvi_label, dtype: int64

In [145]:
adata.obs.scanvi_label.value_counts()

Macrophages                106551
Basal                       93000
Secretory                   67443
AT2                         58398
T cell lineage              43117
Ciliated                    42295
Monocytes                   30395
unlabeled                   24176
Capillary                   20597
Fibroblast lineage          20549
Non-T/B cells               13927
AT1                          9138
Venous                       8449
Dendritic cells              7813
Submucosal Secretory         6988
Mast cells                   6811
B cell lineage               6361
Arterial EC                  6031
Lymphatic EC                 4622
Smooth Muscle                3270
Proliferating cells          1812
Squamous                     1266
Rare                          874
Bronchial Vessel 1            455
Mesothelium                   347
Bronchial Vessel 2            235
KRT5- KRT17+ epithelial        11
Megakaryocytes                 11
Neutrophilic                    2
Name: scanvi_l